<a href="https://colab.research.google.com/github/Gowri-Rk/DeepLearningAssignments/blob/master/Graded%20Assignment%204/DL_GradedAssignment_4_Pytorch_LeNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
import torch

from torch import nn, optim
from torch.autograd import Variable
import torch.nn.functional as F

from sklearn.utils import shuffle
import pandas as pd


In [6]:
from google.colab import  drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
class LeNet(nn.Module):
	def __init__(self):
		super(LeNet, self).__init__()
		self.conv1 = nn.Conv2d(1, 6, (5,5), padding=2)
		self.conv2 = nn.Conv2d(6, 16, (5,5))
		self.fc1   = nn.Linear(16*5*5, 120)
		self.fc2   = nn.Linear(120, 84)
		self.fc3   = nn.Linear(84, 10)
	def forward(self, x):
		x = F.max_pool2d(F.relu(self.conv1(x)), (2,2))
		x = F.max_pool2d(F.relu(self.conv2(x)), (2,2))
		x = x.view(-1, self.num_flat_features(x))
		x = F.relu(self.fc1(x))
		x = F.relu(self.fc2(x))
		x = self.fc3(x)
		return x
	def num_flat_features(self, x):
		size = x.size()[1:]
		num_features = 1
		for s in size:
			num_features *= s
		return num_features

In [8]:
net = LeNet()
print (net)

LeNet(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [9]:
use_gpu = torch.cuda.is_available()
if use_gpu:
	net = net.cuda()
	print ('USE GPU')
else:
	print ('USE CPU')

USE CPU


In [0]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001)

In [13]:
train = pd.read_csv("/content/gdrive/My Drive/Colab Datasets/train.csv").values
train = shuffle(train)
test  = pd.read_csv("/content/gdrive/My Drive/Colab Datasets/test.csv").values

1. Loading data


In [14]:
X_data  = train[:, 1:].reshape(train.shape[0], 1, 28, 28)
X_data  = X_data.astype(float)
X_data /= 255.0
X_data  = torch.from_numpy(X_data);
X_label = train[:,0];
X_label = X_label.astype(int);
X_label = torch.from_numpy(X_label);
X_label = X_label.view(train.shape[0],-1);
print (X_data.size(), X_label.size())

2. Converting data
torch.Size([42000, 1, 28, 28]) torch.Size([42000, 1])


In [15]:
nb_train = train.shape[0]
nb_epoch = 75000
nb_index = 0
nb_batch = 4

3. Training phase


In [19]:
for epoch in range(nb_epoch):
	if nb_index + nb_batch >= nb_train:
		nb_index = 0
	else:
		nb_index = nb_index + nb_batch

	mini_data  = Variable(X_data[nb_index:(nb_index+nb_batch)].clone())
	mini_label = Variable(X_label[nb_index:(nb_index+nb_batch)].clone(), requires_grad = False)
	mini_data  = mini_data.type(torch.FloatTensor)
	mini_label = mini_label.type(torch.LongTensor)
	if use_gpu:
		mini_data  = mini_data.cuda()
		mini_label = mini_label.cuda()
	optimizer.zero_grad()
	mini_out   = net(mini_data)
	mini_label = mini_label.view(nb_batch)
	mini_loss  = criterion(mini_out, mini_label)
	mini_loss.backward()
	optimizer.step() 

	if (epoch + 1) % 2000 == 0:
		print("Epoch = %d, Loss = %f" %(epoch+1, mini_loss.data))

Epoch = 2000, Loss = 2.287052
Epoch = 4000, Loss = 2.310623
Epoch = 6000, Loss = 2.259446
Epoch = 8000, Loss = 1.999218
Epoch = 10000, Loss = 1.120569
Epoch = 12000, Loss = 0.474302
Epoch = 14000, Loss = 0.700304
Epoch = 16000, Loss = 0.081654
Epoch = 18000, Loss = 0.052065
Epoch = 20000, Loss = 0.427617
Epoch = 22000, Loss = 0.873461
Epoch = 24000, Loss = 0.024504
Epoch = 26000, Loss = 0.009882
Epoch = 28000, Loss = 0.165943
Epoch = 30000, Loss = 0.073079
Epoch = 32000, Loss = 0.123819
Epoch = 34000, Loss = 0.029463
Epoch = 36000, Loss = 0.023558
Epoch = 38000, Loss = 0.150513
Epoch = 40000, Loss = 0.003899
Epoch = 42000, Loss = 0.003785
Epoch = 44000, Loss = 0.001223
Epoch = 46000, Loss = 0.019612
Epoch = 48000, Loss = 0.012031
Epoch = 50000, Loss = 0.007315
Epoch = 52000, Loss = 0.026662
Epoch = 54000, Loss = 0.030002
Epoch = 56000, Loss = 0.011724
Epoch = 58000, Loss = 0.005563
Epoch = 60000, Loss = 0.006567
Epoch = 62000, Loss = 0.103720
Epoch = 64000, Loss = 0.201537
Epoch = 6600

In [20]:

Y_data  = test.reshape(test.shape[0], 1, 28, 28)
Y_data  = Y_data.astype(float)
Y_data /= 255.0
Y_data  = torch.from_numpy(Y_data);
print (Y_data.size())
nb_test = test.shape[0]

net.eval()

torch.Size([28000, 1, 28, 28])


LeNet(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

In [22]:
final_prediction = np.ndarray(shape = (nb_test, 2), dtype=int)
for each_sample in range(nb_test):
	sample_data = Variable(Y_data[each_sample:each_sample+1].clone())
	sample_data = sample_data.type(torch.FloatTensor)
	if use_gpu:
		sample_data = sample_data.cuda()
	sample_out = net(sample_data)
	_, pred = torch.max(sample_out, 1)
	final_prediction[each_sample][0] = 1 + each_sample
	final_prediction[each_sample][1] = pred.data
	if (each_sample + 1) % 2000 == 0:
		print("Total tested = %d" %(each_sample + 1))

Total tested = 2000
Total tested = 4000
Total tested = 6000
Total tested = 8000
Total tested = 10000
Total tested = 12000
Total tested = 14000
Total tested = 16000
Total tested = 18000
Total tested = 20000
Total tested = 22000
Total tested = 24000
Total tested = 26000
Total tested = 28000
